In [ ]:
import random
import numpy as np
from tqdm import tqdm
from keras.datasets import mnist

In [ ]:
def standardize(data, ax):
    mean = np.sum(data, axis=ax) / len(data)
    variance = np.sum((data - mean) ** 2) / len(data)
    std_deviation = np.sqrt(variance)

    standardized_data = (data - mean) / std_deviation
    return standardized_data

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [ ]:
train_X = np.array(train_X)
train_y = np.array(train_y)
test_X = np.array(test_X)
test_y = np.array(test_y)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
x_train = train_X.reshape(-1, 28 * 28)
x_train = np.array(standardize(x_train, 0))
x_test = test_X.reshape(-1, 28 * 28)
x_test = np.array(standardize(x_test, 0))
print(x_train.shape, x_test.shape)

(60000, 784) (10000, 784)


In [ ]:

from keras.utils import to_categorical
test_y =np.array(to_categorical(test_y))
print(test_y.shape)
print(test_y[0]) 

(10000, 10)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
# test_y=test_y.reshape(-1, 10)
# print(test_y.shape)

In [ ]:
num_classes=test_y.shape[1]
num_features=x_train.shape[1]
weights = np.zeros((num_classes, num_features))
print(weights.shape)

(10, 784)


In [ ]:
def rand_weights(size):
    dum = list()
    for i in range(size):
        dum.append(random.uniform(-1, 1))
    return np.array(dum)


def sigmoid(x):
    X = [(1 / (1 + np.exp(-z))) for z in x]
    return np.array(X)


def error(H, Y):
    er = 0
    for i in range(len(H)):
        er = er + ((Y[i] * np.log(H[i])) + ((1 - Y[i]) * np.log(1 - H[i])))
    return round(er / len((H)), 4)


def accuracy(class_f, Y_test):
    return (np.sum(class_f == Y_test) / len(Y_test)) * 100


class LogisticRegression():
    def __init__(self, learning_rate=0.05, maxIter=1000, error_ratio=0.01):
        self.__learning_rate = learning_rate
        self.__maxIter = maxIter
        self.__weigths = None
        self.__bias = 0
        self.__error_ratio = error_ratio

    def fit(self, X, Y):
        sample_size = np.array(X).shape[0]
        n_features = np.array(X).shape[1]
        #self.__weigths = np.zeros(n_features)
        self.__weigths = rand_weights(n_features)
        Error = 1
        epoch_bar = tqdm(desc='Epochs', total=self.__maxIter)
        for i in range(self.__maxIter):
            epoch_bar.update(1)
            epoch_bar.set_postfix({'accuracy': f'{1 - Error:.3f}'})
            linear = np.dot(X, self.__weigths) + self.__bias
            prediction = sigmoid(linear)
            dw = (1 / sample_size) * np.dot(X.T, (prediction - Y))
            db = (1 / sample_size) * np.sum(prediction - Y)

            self.__weigths = self.__weigths - self.__learning_rate * dw
            self.__bias = self.__bias - self.__learning_rate * db
            Error = abs(error(prediction, Y))
            if self.__error_ratio > Error:
                break

    def predict(self, X_test):
        linear = np.dot(X_test, self.__weigths) + self.__bias
        Y_predicted = sigmoid(linear)
        class_f = [1 if y > 0.5 else 0 for y in Y_predicted]
        return class_f

    def get_weights(self):
        return self.__weigths

    def set_weights(self, weights):
        self.__weigths = weights

    def get_learning_rate(self):
        return self.__learning_rate

    def set_learning_rate(self, learning_rate):
        self.__learning_rate = learning_rate

In [ ]:
for c in range(num_classes):
    indceis=np.where(train_y==c)
    train_y[:]=0
    train_y[indceis]=1
    model = LogisticRegression()
    model.fit(x_train,train_y)  
    weights[c, :] = model.get_weights()

Epochs:  46%|████▌     | 457/1000 [06:39<07:27,  1.21it/s, accuracy=0.951]

In [ ]:

Z = np.dot(weights, x_test.T)  


softmax_values = np.exp(Z) / np.sum(np.exp(Z), axis=0)


predicted_classes = np.argmax(softmax_values, axis=0)

In [ ]:
accuracy = np.mean(softmax_values == test_y)